In [2]:
# Reading values that my program found and collecting basic statistics

import pandas as pd

df = pd.read_csv(".\\results.csv")

# calculate the total execution time
total_execution_time_sec = round(df['Execution time'].sum(), 2)
minutes = total_execution_time_sec // 60
hours = minutes // 60

print("All STC test were calculated in: {} seconds, {} minutes, hours {}".format(total_execution_time_sec, minutes, hours))

# extract the test name and rename columns
df['Test name'] = df['Test name'].apply(lambda x: x.split('/')[-1])
df = df.rename(columns={'Execution time': 'Execution time Leonid'})
df = df.rename(columns={'Result': 'Result Leonid'})

df

All STC test were calculated in: 709.79 seconds, 11.0 minutes, hours 0.0


,Test name,Execution time Leonid,Result Leonid,Values
0,STC_0001.smt2,0.003764,SAT,x = -100 y = 1 z = 100
1,STC_0002.smt2,0.323468,SAT,x = -47 y = -24 z = 49
2,STC_0003.smt2,0.573846,SAT,x = -5 y = 4 z = 4
3,STC_0004.smt2,1.253370,Not SAT,NaN
4,STC_0005.smt2,1.565967,Not SAT,NaN
...,...,...,...,...
995,STC_0996.smt2,0.549865,SAT,x = -7 y = 2 z = 11
996,STC_0997.smt2,0.556982,SAT,x = -3 y = 8 z = 8
997,STC_0998.smt2,0.533815,SAT,x = -9 y = -1 z = 12
998,STC_0999.smt2,0.530791,SAT,x = -9 y = 0 z = 12


In [3]:
# Reading values from original cvc5 executable and collecting basic statistics
import pandas as pd

df_orig = pd.read_csv(".\\results_original_cvc5.csv")

# calculate the total execution time
total_execution_time_sec = round(df_orig['Execution time'].sum(), 2)
minutes = total_execution_time_sec // 60
hours = minutes // 60

print("All STC test were calculated in: {} seconds, {} minutes, hours {}".format(total_execution_time_sec, minutes, hours))
print("Inmortant note, that timeout was set to 1 minute!")

# extract the test name and rename columns
df_orig['Test name'] = df_orig['Test name'].apply(lambda x: x.split('/')[-1])
df_orig = df_orig.rename(columns={'Execution time': 'Execution time CVC5'})
df_orig = df_orig.rename(columns={'Result': 'Result CVC5'})


df_orig

All STC test were calculated in: 55544.7 seconds, 925.0 minutes, hours 15.0
Inmortant note, that timeout was set to 1 minute!


,Test name,Execution time CVC5,Result CVC5,Values
0,STC_0001.smt2,22.307753,sat,NaN
1,STC_0002.smt2,0.249582,sat,NaN
2,STC_0003.smt2,0.378245,sat,NaN
3,STC_0004.smt2,60.026129,NaN,NaN
4,STC_0005.smt2,60.028352,NaN,NaN
...,...,...,...,...
995,STC_0996.smt2,60.024707,NaN,NaN
996,STC_0997.smt2,60.023573,NaN,NaN
997,STC_0998.smt2,60.024111,NaN,NaN
998,STC_0999.smt2,60.030542,NaN,NaN


In [4]:
# Mmerging 2 datframes to compare and make analysis

resulting_df = pd.merge(df, df_orig, on='Test name')

# Droping values calculaed from CVC5, because they are not calculated
resulting_df = resulting_df.drop('Values_y', axis=1)

# Values calculated by my solver could be used later for verification
resulting_df = resulting_df.rename(columns={'Values_x': 'Values Leonid'})

# strip any leading or trailing whitespace from the "Result CVC5" column
resulting_df['Result Leonid'] = resulting_df['Result Leonid'].str.strip().str.lower()

# count the number of rows where the "Result Leonid" column and the "Result CVC5" column have the same value
same_results_counter = sum(resulting_df['Result Leonid'] == resulting_df['Result CVC5'])
print("In total there are {} cases when the programs gave the same results".format(same_results_counter))


# Define the lambda function to calculate missing values
calculate_missing = lambda col: col.isna().sum()

# Apply the lambda function to column 'B'
missing_values_cvc = resulting_df['Result CVC5'].pipe(calculate_missing)

print("There are {} missing values in column Result CVC5.".format(missing_values_cvc))

resulting_df


In total there are 144 cases when the programs gave the same results
There are 856 missing values in column Result CVC5.


,Test name,Execution time Leonid,Result Leonid,Values Leonid,Execution time CVC5,Result CVC5
0,STC_0001.smt2,0.003764,sat,x = -100 y = 1 z = 100,22.307753,sat
1,STC_0002.smt2,0.323468,sat,x = -47 y = -24 z = 49,0.249582,sat
2,STC_0003.smt2,0.573846,sat,x = -5 y = 4 z = 4,0.378245,sat
3,STC_0004.smt2,1.253370,not sat,NaN,60.026129,NaN
4,STC_0005.smt2,1.565967,not sat,NaN,60.028352,NaN
...,...,...,...,...,...,...
995,STC_0996.smt2,0.549865,sat,x = -7 y = 2 z = 11,60.024707,NaN
996,STC_0997.smt2,0.556982,sat,x = -3 y = 8 z = 8,60.023573,NaN
997,STC_0998.smt2,0.533815,sat,x = -9 y = -1 z = 12,60.024111,NaN
998,STC_0999.smt2,0.530791,sat,x = -9 y = 0 z = 12,60.030542,NaN


## Report on Comparison of STC Test Results: My SMT Solver vs. Original CVC5 Solver

### Introduction:

I developed an SMT solver that solves constraints by testing values in the range from -100 to 100. I used this solver to run 1000 STC tests and compared my results to those of the original CVC5 solver. In this report, I will discuss the results of this comparison.

### Comparison of Results:

Out of the 1000 tests, my program gave the same result as the original CVC5 solver in 144 cases. This suggests that my solver is performing reasonably well, given that it is based on a brute-force approach and tests values only in a limited interval.

### Time Comparison:

The time it took for my solver to run all 1000 tests was 11 minutes, while the original CVC5 solver took 15 hours. However, I set a timeout of 1 minute for the original CVC5 solver, after which it terminated if a value was not found. This indicates that my solver is faster than the original CVC5 solver in finding values for the range tested.

### Conclusion:

In conclusion, the results of the comparison show that my SMT solver is performing well, given its brute-force approach and limited testing range. Additionally, my solver is faster than the original CVC5 solver in finding values within the range tested. Also from dataframe it is clear that 144 tests were calculated in 1 minute, and the rest 856 tests need more time. However, it is important to note that further testing is required to evaluate the performance of my solver on more complex problems and larger testing ranges. 
